In [4]:
import pandas as pd
import json
import numpy as np
# prod_arg = ['category', 'description', 'title', 'brand', 'main_cat', 'price', 'asin', 'imageURL', 'imageURLHighRes']
# rat_arg=['overall', 'reviewerID','asin', 'reviewerName']
rating_df = pd.read_json('grocery_rating.json', lines=True)
# rating_df.rename(columns={"0006564224":"asin","A3NSN9WOX8470M":"reviewerID","5.0":"overall"}, inplace=True)
# rating_df = rating_df[['overall', 'reviewerID','asin', 'reviewerName']]
products_df = pd.read_json('meta_grocery.json', lines=True)
products_df = products_df[['category', 'description', 'title', 'brand', 'main_cat', 'price', 'asin', 'imageURL', 'imageURLHighRes']]

In [5]:
print(rating_df.shape)
print(products_df.shape)
rating_df.tail(5)

(1143860, 12)
(287051, 9)


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
1143855,5,True,"09 8, 2017",A223YRQH2Z5T1D,B01HJF6FRA,flint5292,"As a new vegan, it is sometimes difficult to r...","As a new vegan, it is sometimes difficult to ...",1504828800,4,NaN,NaN
1143856,5,True,"08 4, 2017",A38GDA4TB9EILT,B01HJF6FRA,Moriah Bolyard,The best thing ever is ordering a product you ...,The best thing ever is ordering a product you ...,1501804800,3,NaN,NaN
1143857,5,True,"07 4, 2017",A2025PN7HDC5BO,B01HJF6FRA,M.C,I used to love ranch before I became vegan. It...,Just what the vegan ordered!,1499126400,5,NaN,NaN
1143858,5,True,"06 7, 2017",A1NY7XWC7EPQOA,B01HJF6FRA,Greensboro,I cannot have dairy nor gluten. This is as cl...,This is as close to Ranch as I will ever be ab...,1496793600,2,NaN,NaN
1143859,3,True,"05 20, 2017",A1P0X9E6F99J4T,B01HJF6FRA,Qoyllor,Needs improvement to make it taste like real r...,So so,1495238400,NaN,NaN,NaN


In [6]:
rating_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143860 entries, 0 to 1143859
Data columns (total 12 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   overall         1143860 non-null  int64 
 1   verified        1143860 non-null  bool  
 2   reviewTime      1143860 non-null  object
 3   reviewerID      1143860 non-null  object
 4   asin            1143860 non-null  object
 5   reviewerName    1143722 non-null  object
 6   reviewText      1143470 non-null  object
 7   summary         1143641 non-null  object
 8   unixReviewTime  1143860 non-null  int64 
 9   vote            158202 non-null   object
 10  style           592086 non-null   object
 11  image           9510 non-null     object
dtypes: bool(1), int64(2), object(9)
memory usage: 97.1+ MB


# Popularity based recommendations

In [7]:
rat_count_df=rating_df.groupby('asin').count()['overall'].reset_index()
rat_count_df.rename(columns={"overall":"rating-count"}, inplace=True)
rat_count_df

,asin,rating-count
0,4639725043,30
1,4639725183,12
2,5463213682,9
3,9742356831,96
4,B00004S1C5,14
...,...,...
41315,B01HJ2OFYI,12
41316,B01HJ4FWGG,18
41317,B01HJ7QPKU,8
41318,B01HJCUNOO,10


In [8]:
rat_avg_df = rating_df.groupby('asin').mean()['overall'].reset_index()
rat_avg_df.rename(columns={"overall":"rating-avg"}, inplace=True)
rat_avg_df


C:\Users\surya\AppData\Local\Temp\ipykernel_11412\3651525842.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  rat_avg_df = rating_df.groupby('asin').mean()['overall'].reset_index()


,asin,rating-avg
0,4639725043,4.200000
1,4639725183,4.833333
2,5463213682,4.888889
3,9742356831,4.552083
4,B00004S1C5,4.000000
...,...,...
41315,B01HJ2OFYI,4.416667
41316,B01HJ4FWGG,4.444444
41317,B01HJ7QPKU,4.250000
41318,B01HJCUNOO,4.400000


In [12]:
popular_df = rat_avg_df.merge(rat_count_df,on="asin")
popular_df = popular_df[popular_df['rating-count'] > 200].sort_values('rating-count', ascending=False).reset_index()
popular_df

,index,asin,rating-avg,rating-count
0,24039,B00BUKL666,4.585353,7387
1,21028,B008QMX2SG,4.573539,6228
2,25477,B00D3M2QP4,4.573702,6221
3,34636,B00R7PWK7W,4.568645,3387
4,5781,B000X3TPHS,4.756436,3030
...,...,...,...,...
706,16543,B00555H0LC,4.393035,201
707,2244,B000FFIEL2,4.393035,201
708,21720,B009GX9TSG,4.537313,201
709,20957,B008JA73RG,4.517413,201


In [13]:
best_products = products_df.merge(popular_df,on="asin").sort_values('rating-count', ascending=False).reset_index()
# display(best_products.head(10))
print(best_products.shape)
best_products['title'].drop_duplicates(inplace=True)
best_products

(727, 13)


,level_0,category,description,title,brand,main_cat,price,asin,imageURL,imageURLHighRes,index,rating-avg,rating-count
0,560,"[Grocery & Gourmet Food, Snack Foods, Bars, Nu...",[These bars are where our journey started and ...,"KIND Bars, Dark Chocolate Nuts &amp; Sea Salt,...",KIND,Grocery,$13.67,B00BUKL666,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,24039,4.585353,7387
1,526,"[Grocery & Gourmet Food, Snack Foods, Bars, Nu...",[These bars are where our journey started and ...,"Kind Bars, Madagascar Vanilla Almond, Gluten F...",KIND,Grocery,$14.79,B008QMX2SG,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,21028,4.573539,6228
2,576,"[Grocery & Gourmet Food, Breakfast Foods, Brea...",[These bars are where our journey started and ...,"KIND Bars, Dark Chocolate Chili Almond, Gluten...",KIND,Grocery,$15.53,B00D3M2QP4,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,25477,4.573702,6221
3,676,"[Grocery & Gourmet Food, Snack Foods, Bars, Nu...",[These bars are where our journey started and ...,"KIND Bars, Caramel Almond and Sea Salt, Gluten...",KIND,Grocery,$13.67,B00R7PWK7W,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,34636,4.568645,3387
4,198,"[Grocery & Gourmet Food, Candy & Chocolate, Ha...",[Our pops are tops! YumEarth Organic Lollipops...,"YumEarth Organic Lollipops, Assorted Flavors, ...",YumEarth,Grocery,,B000X3TPHS,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,5781,4.756436,3030
...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,457,"[Grocery & Gourmet Food, Jams, Jellies & Sweet...","[<div class=""boost-aplus-container""> <div clas...","Chocolate Hazelnut Butter by Justin's, Organic...",Justin's Nut Butter,Grocery,$9.99,B00555H0LC,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,16543,4.393035,201
723,201,"[Grocery & Gourmet Food, Breakfast Foods, Cere...","[Trust, honesty and integrity. These are the t...","Bob's Red Mill, Quick Rolled Oats, Organic, 32 oz",Bob's Red Mill,Grocery,,B000YF9944,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,5901,4.810945,201
724,543,"[Grocery & Gourmet Food, Olives, Pickles & Rel...",[Pitted snack olives with a hint of garlic tha...,"Mario Camacho Foods Pitted Snack Olives, Seaso...",Mario Camacho,Grocery,$11.22,B009GX9TSG,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,21720,4.537313,201
725,523,"[Grocery & Gourmet Food, Beverages, Bottled Be...",[Steady energy from green tea. Thats what V8 +...,"V8 +Energy, Juice Drink with Green Tea, Peach ...",V8,Grocery,$15.92,B008JA73RG,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,20957,4.517413,201


# ITEM - ITEM collaborative filtering

In [14]:
temp_y = rating_df.groupby('reviewerID').count()['overall'] > 30
customerbyvote = temp_y[temp_y].index
customerbyvote

Index(['A100RH4M1W1DF0', 'A100UD67AHFODS', 'A100WO06OQR8BQ', 'A100ZQDV7L8PVV',
       'A1047EDJ84IMAS', 'A1080SE9X3ECK0', 'A10AFVU66A79Y1', 'A10AV9H1SI1YAV',
       'A10B6G6W3DW1EY', 'A10CRW7XRJBJ2G',
       ...
       'AZ7VAUBDK6V0P', 'AZ80HZOJPR17R', 'AZ8XSDMIX04VJ', 'AZA595ZPIG240',
       'AZKV4KTSLZFKN', 'AZOCADWL9N2H2', 'AZPL19WYM24G9', 'AZPS09VLXJDQP',
       'AZU8GQQW6HESP', 'AZV26LP92E6WU'],
      dtype='object', name='reviewerID', length=2505)

In [15]:
filtered_user = rating_df[rating_df['reviewerID'].isin(customerbyvote)]
filtered_user

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
26,4,True,"01 18, 2015",ALOGZDSPWZI72,4639725043,S. Thornburg,"Good tea flavor. Not the best I've ever had, b...",Good flavor,1421539200,NaN,NaN,NaN
27,3,True,"12 17, 2014",AO15IC5EEMS8P,4639725043,Alice Pollack,The tea is wonderful. I can't find it in any r...,The tea is wonderful. I can't find it in any r...,1418774400,NaN,NaN,NaN
29,5,True,"12 7, 2014",A1CT6MLYNBG3L4,4639725043,Expo Reviews,This is the best flavor tea for iced tea I hav...,Great for Iced Tea,1417910400,2,NaN,NaN
31,5,False,"04 22, 2014",A1WOMXOEX90ARI,4639725043,Avid Camper,This is NOT the regular Lipton's Tea sold in t...,Very Good Tea...,1398124800,NaN,NaN,NaN
32,5,True,"04 14, 2014",A12FEHYJOJZZRY,4639725043,J. Voelkle,We discovered 'Yellow Label' tea while in S.E....,"Fresh, strong tea",1397433600,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1143779,5,True,"06 27, 2018",A1L4IVKI6Y5R2M,B01HDR1RX6,HAPPYORSAD,VERY GOOD FOR US CARBAHOLICS TRYING TO NOT EAT...,IT'S GOOD!,1530057600,NaN,NaN,NaN
1143785,5,False,"05 21, 2018",A12ENBT314RFXR,B01HEDNPFM,Patricia Ferdig,These lasted us for so long! My husband took m...,Perfect for the office drawer for late afterno...,1526860800,NaN,"{'Size:': ' 40 Count', 'Style:': ' Coffee/Cream'}",NaN
1143789,5,True,"03 21, 2018",A1I4NHYX40MFD,B01HFCI980,Cheryl,I have always hated the taste of carob but cho...,Doesn't taste like carob at all!,1521590400,NaN,"{'Size:': ' 1Lb', 'Flavor:': ' Sweetened'}",NaN
1143835,5,True,"03 6, 2018",A2LUY8LI3VB1D1,B01HHXH6WM,R.A.,"Love the taste, just enough pickling flavour. ...","Delicious, a constant in my fridge.",1520294400,NaN,NaN,NaN


In [16]:
temp_x = filtered_user.groupby('asin').count()['overall'] > 50
products_gt_50 = temp_x[temp_x].index
products_gt_50

Index(['B00014JNI0', 'B000168QTU', 'B0001H22B2', 'B0001LO3FG', 'B00028MOQS',
       'B00061EOVO', 'B00061ETX2', 'B0007LXU0Y', 'B0007R9L4M', 'B00099XK6I',
       ...
       'B00XA8XWGS', 'B00XABKAWY', 'B00XOORKRK', 'B0105E0TEM', 'B010ULFOWC',
       'B013IG9KVW', 'B0179XFQ5E', 'B01948LFD4', 'B01BLAW370', 'B01E5XTW24'],
      dtype='object', name='asin', length=184)

In [17]:
filtered_product = filtered_user[filtered_user['asin'].isin(products_gt_50)]
filtered_product = filtered_product.merge(products_df, on='asin')[['asin', 'overall', 'reviewerID', 'title', 'price', 'category', 'imageURLHighRes']]
filtered_product

,asin,overall,reviewerID,title,price,category,imageURLHighRes
0,B00014JNI0,5,A3TC22WXURSRI,YS Organic Bee Farms CERTIFIED ORGANIC RAW HON...,$17.44,"[Grocery & Gourmet Food, Cooking & Baking, Syr...",[]
1,B00014JNI0,5,A3TC22WXURSRI,YS Organic Bee Farms CERTIFIED ORGANIC RAW HON...,$17.44,"[Grocery & Gourmet Food, Cooking & Baking, Syr...",[]
2,B00014JNI0,5,ALOGZDSPWZI72,YS Organic Bee Farms CERTIFIED ORGANIC RAW HON...,$17.44,"[Grocery & Gourmet Food, Cooking & Baking, Syr...",[]
3,B00014JNI0,5,ALOGZDSPWZI72,YS Organic Bee Farms CERTIFIED ORGANIC RAW HON...,$17.44,"[Grocery & Gourmet Food, Cooking & Baking, Syr...",[]
4,B00014JNI0,5,A2B8SB10Z6IG73,YS Organic Bee Farms CERTIFIED ORGANIC RAW HON...,$17.44,"[Grocery & Gourmet Food, Cooking & Baking, Syr...",[]
...,...,...,...,...,...,...,...
17353,B01E5XTW24,4,A3NVNHJ1SJW080,"Green Mountain Coffee, Breakfast Blend Decaf, ...",$41.58,"[Grocery & Gourmet Food, Beverages, Coffee, Te...",[]
17354,B01E5XTW24,5,A3N3TRD4YGTKUY,"Green Mountain Coffee, Breakfast Blend Decaf, ...",$41.58,"[Grocery & Gourmet Food, Beverages, Coffee, Te...",[]
17355,B01E5XTW24,5,ALJ1S9LOYB08X,"Green Mountain Coffee, Breakfast Blend Decaf, ...",$41.58,"[Grocery & Gourmet Food, Beverages, Coffee, Te...",[]
17356,B01E5XTW24,4,A10PEXB6XAQ5XF,"Green Mountain Coffee, Breakfast Blend Decaf, ...",$41.58,"[Grocery & Gourmet Food, Beverages, Coffee, Te...",[]


In [18]:
pivottable = filtered_product.pivot_table(index='asin', columns='reviewerID', values='overall')
pivottable.fillna(0,inplace=True)
pivottable

reviewerID,A100RH4M1W1DF0,A100UD67AHFODS,A100WO06OQR8BQ,A1047EDJ84IMAS,A1080SE9X3ECK0,A10AFVU66A79Y1,A10AV9H1SI1YAV,A10B6G6W3DW1EY,A10CRW7XRJBJ2G,A10EGQ9E4RNAH9,...,AZ6E6PR5QH0K,AZ7VAUBDK6V0P,AZ80HZOJPR17R,AZ8XSDMIX04VJ,AZA595ZPIG240,AZOCADWL9N2H2,AZPL19WYM24G9,AZPS09VLXJDQP,AZU8GQQW6HESP,AZV26LP92E6WU
asin,,,,,,,,,,,,,,,,,,,,,
B00014JNI0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
B000168QTU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0001H22B2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0001LO3FG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
B00028MOQS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B013IG9KVW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0179XFQ5E,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B01948LFD4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_dist=cosine_similarity(pivottable)
cosine_dist

array([[1.        , 0.02414958, 0.        , ..., 0.01095302, 0.0218558 ,
        0.05136238],
       [0.02414958, 1.        , 0.07004282, ..., 0.04802341, 0.0141218 ,
        0.04718214],
       [0.        , 0.07004282, 1.        , ..., 0.01906072, 0.06663557,
        0.02887729],
       ...,
       [0.01095302, 0.04802341, 0.01906072, ..., 1.        , 0.        ,
        0.        ],
       [0.0218558 , 0.0141218 , 0.06663557, ..., 0.        , 1.        ,
        0.        ],
       [0.05136238, 0.04718214, 0.02887729, ..., 0.        , 0.        ,
        1.        ]])

In [22]:
def recommender(item_name):
    index = np.where(pivottable.index==item_name)[0][0]
    similar = list(enumerate(cosine_dist[index]))
    similar.sort(key=lambda a:a[1], reverse=True)
    data=[]
    for i in similar:
#         item=[]
#         temp_df=popular_df[best_products['asin']==pivottable.index[i[0]]]
#         item.extend(list(temp_df.drop_duplicates('title')['asin'].values))
#         item.extend(list(temp_df.drop_duplicates('title')['rating-avg'].values))
#         item.extend(list(temp_df.drop_duplicates('title')['rating-count'].values))
#         data.append(item)
        k=pivottable.index[i[0]]
        print('*'*100)
        print(k)
        print(best_products.loc[best_products['asin'] == k]['title'].to_string())

#     return data

In [23]:
item = "B0001LO3FG"
print(best_products.loc[best_products['asin'] == item]['title'].to_string())
print("=============================")
recommender(item)

5    Twinings of London Pure Camomile Herbal Tea Ba...
6    Twinings of London Pure Camomile Herbal Tea Ba...
****************************************************************************************************
B0001LO3FG
5    Twinings of London Pure Camomile Herbal Tea Ba...
6    Twinings of London Pure Camomile Herbal Tea Ba...
****************************************************************************************************
B000F4DKAI
7    Twinings of London English Afternoon Black Tea...
****************************************************************************************************
B00XA8XWGS
14    Twinings of London Christmas Black Tea Bags, 2...
****************************************************************************************************
B000168QTU
23    Celestial Seasonings Tension Tamer Tea, 20 ct
24    Celestial Seasonings Tension Tamer Tea, 20 ct
****************************************************************************************************
B001N2GRX8
57   

In [24]:
popular_df

,index,asin,rating-avg,rating-count
0,24039,B00BUKL666,4.585353,7387
1,21028,B008QMX2SG,4.573539,6228
2,25477,B00D3M2QP4,4.573702,6221
3,34636,B00R7PWK7W,4.568645,3387
4,5781,B000X3TPHS,4.756436,3030
...,...,...,...,...
706,16543,B00555H0LC,4.393035,201
707,2244,B000FFIEL2,4.393035,201
708,21720,B009GX9TSG,4.537313,201
709,20957,B008JA73RG,4.517413,201


In [25]:
import pickle
pickle.dump(popular_df, open('popular_df_g.pkl','wb'))

In [26]:
import pickle
pickle.dump(best_products, open('best_products_g.pkl','wb'))

In [27]:
import pickle
pickle.dump(cosine_dist, open('cosine_dist_g.pkl','wb'))

In [28]:
import pickle
pickle.dump(pivottable, open('pivottable_g.pkl','wb'))